<a href="https://colab.research.google.com/github/awaiskhan005/DEEP-LEARNING-AND-FORECASTING/blob/main/DEEP_SEEK_franklinn_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Required Libraries
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Install the 'ta' library if it's not already installed
!pip install ta

# Install the 'python-binance' library if it's not already installed
!pip install python-binance # This line is added to install the missing library

# Now import from 'ta'
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from binance.client import Client # This import should now work
from datetime import datetime, timedelta, timezone
import time

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=30d26a04ec7447bbe412d09d67fef73b96f5c797e26ac2d67927c341cf350e14
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.5 MB/s eta 0:00:00


In [3]:
from binance.client import Client

# Binance API keys
api_key = "8gexdTyvvgmMQCqeFH9Uw4kDUc7xdIQ7a8ZGYxIsczMFUD78WkwetcNQYNbKPe7c"
secret_key = "g39iujh5129t6GcMoCYIGXKusnfkE9PMpcZVyNNk1XIOIg2eGumGM6fb9za6DOZ"

# Initialize Binance client
client = Client(api_key, secret_key)

BinanceAPIException: APIError(code=0): Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import joblib

# Fetch 6-hour candle data
def fetch_candles(symbol, interval, lookback):
    candles = client.get_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(candles, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    # Convert 'open', 'high', 'low', 'close' columns to numeric
    for col in ['open', 'high', 'low', 'close']:
        df[col] = pd.to_numeric(df[col]) # Convert to numeric to avoid TypeError

    return df

# Feature engineering
def feature_engineering(df):
    # Add features like moving averages, RSI, etc.
    df['returns'] = df['close'].pct_change()
    df['ma_10'] = df['close'].rolling(window=10).mean()
    df['ma_50'] = df['close'].rolling(window=50).mean()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    df.dropna(inplace=True)
    return df

# Train and save models
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT']
for symbol in symbols:
    # Fetch data
    df = fetch_candles(symbol, Client.KLINE_INTERVAL_6HOUR, '500 days ago UTC')
    df = feature_engineering(df)

    # Prepare features and target
    X = df[['ma_10', 'ma_50', 'rsi']]
    y = np.where(df['returns'].shift(-1) > 0, 1, 0)  # Predict if the next return is positive

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = XGBClassifier()
    model.fit(X_train, y_train)

    # Save model
    joblib.dump(model, f"{symbol}_xgboost_model.pkl")
    print(f"Model saved for {symbol}")

In [ ]:
import joblib

# Load models and predict
for symbol in symbols:
    # Load model
    model = joblib.load(f"{symbol}_xgboost_model.pkl")

    # Fetch latest data
    # Increase the lookback period to have enough data for feature engineering
    df = fetch_candles(symbol, Client.KLINE_INTERVAL_6HOUR, '7 days ago UTC')
    df = feature_engineering(df)

    # Check if the DataFrame is empty after feature engineering
    if df.empty:
        print(f"DataFrame for {symbol} is empty after feature engineering. Skipping prediction.")
        continue

    # Prepare features
    X = df[['ma_10', 'ma_50', 'rsi']].iloc[-1].values.reshape(1, -1)

    # Predict probabilities
    probabilities = model.predict_proba(X)
    percentage_chances = probabilities * 100

    print(f"{symbol}: Probability of increase = {percentage_chances[0][1]:.2f}%")

In [ ]:
def execute_buy_order(symbol, amount_usdt):
    try:
        order = client.order_market_buy(
            symbol=symbol,
            quantity=amount_usdt / float(client.get_symbol_ticker(symbol=symbol)['price'])
        )
        print(f"Buy order executed for {symbol}: {order}")
    except Exception as e:
        print(f"Error executing buy order for {symbol}: {e}")

# Main loop
for symbol in symbols:
    # Load model
    model = joblib.load(f"{symbol}_xgboost_model.pkl")

    # Fetch latest data
    df = fetch_candles(symbol, Client.KLINE_INTERVAL_6HOUR, '1 day ago UTC')
    df = feature_engineering(df)

    # Prepare features
    X = df[['ma_10', 'ma_50', 'rsi']].iloc[-1].values.reshape(1, -1)

    # Predict probabilities
    probabilities = model.predict_proba(X)
    percentage_chances = probabilities * 100

    # Execute buy order if probability > 60%
    if percentage_chances[0][1] > 60:
        print(f"Buy signal detected for {symbol} (Probability = {percentage_chances[0][1]:.2f}%)")
        execute_buy_order(symbol, amount_usdt=100)  # Adjust amount as needed